## Key points

1. Pernyataan masalah
2. Memahami Dataset Churn Prediction
3. Data Preprocessing Steps
4. EDA ( Exploration Data Analysis )
5. Model building
6. Model Evaluation

# 1. Pernyataan Masalah

   Kita diberikan dataset nasabah bank, dan kita ingin membuat model prekdiksi churn yang dapat mengklarifikasikan nasabah menjadi dua kelas:
   + Candidat Chrun 
   + Non Churn

    "Problem ini akan kita selesaikan dengan pendekatan supervise learning, artinya kita akan memberikan label  sample data input dan output".

# 2. Memahami dataset Churn prediction

Source dataset diambil dari ==> https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers

+ RowNumber — corresponds to the record (row) number and has no effect on the output.
+ CustomerId — contains random values and has no effect on customer leaving the bank.
+ Surname — the surname of a customer has no impact on their decision to leave the bank.
+ CreditScore — can have an effect on customer churn, since a customer with a higher credit score is less likely to leave the bank.
+ Geography — a customer’s location can affect their decision to leave the bank.
+ Gender — it’s interesting to explore whether gender plays a role in a customer leaving the bank.
+ Age — this is certainly relevant, since older customers are less likely to leave their bank than younger ones.
+ Tenure — refers to the number of years that the customer has been a client of the bank. Normally, older clients are more loyal and less likely to leave a bank.

    + Balance—also a very good indicator of customer churn, as people with a higher balance in their accounts are less likely to leave the bank compared to those with lower balances.
    + NumOfProducts—refers to the number of products that a customer has purchased through the bank.
    + HasCrCard—denotes whether or not a customer has a credit card. This column is also relevant, since people with a credit card are less likely to leave the bank.
    + IsActiveMember—active customers are less likely to leave the bank.
    + EstimatedSalary—as with balance, people with lower salaries are more likely to leave the bank compared to those with higher salaries.
    + Exited—whether or not the customer left the bank.

In [ ]:
# pip install lightgbm

In [ ]:
# Import library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno # provides a small toolset of flexible and easy-to-use missing data visualizations 
                         # and utilities that allows you to get a quick visual summary of the completeness 
                         # (or lack thereof) of your dataset.
%matplotlib inline

from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV,train_test_split,cross_val_score
import itertools
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import LocalOutlierFactor # çok değişkenli aykırı gözlem incelemesi
from sklearn.preprocessing import scale,StandardScaler, MinMaxScaler,Normalizer,RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import  accuracy_score, f1_score, precision_score,confusion_matrix, recall_score, roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
import plotly.graph_objects as go #
from plotly.subplots import make_subplots
import plotly.express as px  # digunakan untuk keperluan visualisasi data


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

%config InlineBackend.figure_format = 'retina'

# to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);  # to display all columns and rows
pd.set_option('display.float_format', lambda x: '%.2f' % x) # The number of numbers that will be shown after the comma.

# Load Dataset

In [ ]:
# Load dataset

df = pd.read_csv('/Users/mac/Study/Study_burhanudin/Common_Data/churn.csv')


# Kita amati info dataset kita
df.info()

In [ ]:
# Kita lihat sekilas data terseut
df.head()

+ Dapat di simpulkan bahwa :
    + Rownumber, CustomerId dan Surename hanya mendeskripsikan pelanggan secara unik dan tidak menpengaruhi variable target, sehingga kita dapat menghapus kolom ini.
    + Kita memiliki dua kolom category 'geografi' dan 'gender' yang akan kami encode menggunakan labelencoder()

In [ ]:
df.info()

In [ ]:
# Selanjutnya akan menampilkan nama-nama kolom pada dataframe
df.columns.values

## Visualize missing values

In [ ]:
# Tampilkan missing value dalam bentik matrix
msno.matrix(df)

+ Sekilas tampak tidak ada null values

## Visualize data

In [ ]:
g_labels = ['Male', 'Female']
c_labels = ['Non Churn(0)', 'Candidat Chrun(1)']
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=g_labels, values=df['Gender'].value_counts(), name="Gender"),
              1, 1)
fig.add_trace(go.Pie(labels=c_labels, values=df['Exited'].value_counts(), name="Exited"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.3, hoverinfo="label+percent+name", textfont_size=16)
fig.update_layout(
    title_text="Gender and Exited Distributions",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Gender', x=0.18, y=0.5, font_size=18, showarrow=False),
                 dict(text='Exited', x=0.81, y=0.5, font_size=20, showarrow=False)])
fig.show()

+ 20,4% nasabah berpindah ke perusahaan bank yang lain
+ Laki laki lebih menominasi dari sisi kepelangganan layanan telekomunikasi ini sebesar 54,6%

In [ ]:
# Kita akan hitung secara angka berapa banyak pelanggan yang masih bertahan ( No Churn) 
# di urutkan berdasarkan jenis kelamin

df['Exited'][df['Exited']==0].groupby(by=df['Gender']).count()

In [ ]:
# Kita juga akan hitung secara angka berapa banyak pelanggan yang churn 
# di urutkan berdasarkan jenis kelamin

df['Exited'][df['Exited']==1].groupby(by=df['Gender']).count()

In [ ]:
# Kita visualize kan gabungan data pelanggan tersebut

plt.figure(figsize=(8, 8))
labels =["Exited: Candidat Chrun","Exited:Non Churn"]
values = [2037,7963]
labels_gender = ["F","M","F","M"]
sizes_gender = [1139,898 , 3404,4559]
colors = ['#ff6666', '#66b3ff']
colors_gender = ['#c2c2f0','#ffb3e6', '#c2c2f0','#ffb3e6']
explode = (0.3,0.3) 
explode_gender = (0.1,0.1,0.1,0.1)
textprops = {"fontsize":15}
#Plot
plt.pie(values, labels=labels,autopct='%1.1f%%',pctdistance=1.08, labeldistance=0.8,colors=colors, startangle=90,frame=True, explode=explode,radius=10, textprops =textprops, counterclock = True, )
plt.pie(sizes_gender,labels=labels_gender,colors=colors_gender,startangle=90, explode=explode_gender,radius=7, textprops =textprops, counterclock = True, )
#Draw circle
centre_circle = plt.Circle((0,0),5,color='black', fc='white',linewidth=0)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('Churn Distribution w.r.t Gender: Male(M), Female(F)', fontsize=24, y=1.1)

# show plot 
 
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# Kita lihat distribusi data customer jika ditinjauan Jumlah produk layanan perbankan yang di ambil 

fig = px.histogram(df, x="Exited", color="NumOfProducts", barmode="group", title="<b>Customer Number Of Product distribution<b>")
fig.update_layout(width=600, height=500, bargap=0.1)
fig.show()

# 3. EDA (Exploratory of Data Analysis)

## 3.1 Data Preprocessing steps

In [ ]:
dependent_variable_name = "Exited"

In [ ]:
def data_prepare():
    df_prep = df.copy()
    
    
    missing_value_len = df.isnull().any().sum()
    if missing_value_len == 0:
        print("No Missing Value")
    else:
        print("Investigate Missing Value, Missing Value : " + str(missing_value_len))
    print("\n")
    
    show_unique_count_variables(df = df_prep)
    
    
    df_prep['Tenure'] =  df_prep.Tenure.astype(np.float)
    df_prep['NumOfProducts'] =  df_prep.NumOfProducts.astype(np.float)
    return df_prep

### Representasi data nilai unik pada semua variabel:

In [ ]:
def show_unique_count_variables(df):
    for index, value in df.nunique().items():
        print(str(index) + "\n\t\t\t:" + str(value))

## 3.2. Outliers Observe (LOF method and Supress)

#### 3.2.1. Outlier Editing

In [ ]:
def show_outliers(df):
    
# mengamati fitur outlier dengan melihat semua kolom dari atas

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num_cols = df.select_dtypes(include=numerics)
    sns.set(font_scale = 0.7) 
    fig, axes = plt.subplots(nrows = 2, ncols = 5, gridspec_kw =  dict(hspace=0.3), figsize = (12,9))
    fig.tight_layout()
    for ax,col in zip(axes.flatten(), df_num_cols.columns):
        sns.boxplot(x = df_num_cols[col], color='green', ax = ax)
    fig.suptitle('Observing Outliers', color = 'r', fontsize = 14)

#### 3.2.2. Visualization of outliers according to the LOF method

In [ ]:
def lof_observation(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num_cols = df.select_dtypes(include=numerics)
    df_outlier = df_num_cols.astype("float64")
    clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
    clf.fit_predict(df_outlier)
    df_scores = clf.negative_outlier_factor_
    scores_df = pd.DataFrame(np.sort(df_scores))
    
    scores_df.plot(stacked=True, xlim = [0,20], color='r', title='Visualization of outliers according to the LOF method', style = '.-');                # first 20 observe
    th_val = np.sort(df_scores)[2]
    outliers = df_scores > th_val
    df = df.drop(df_outlier[~outliers].index)
    df.shape
    return df

#### 3.2.3 Outiler suppression

In [ ]:
def clear_outliers(df):
    
    # Penekanan credit Score
    
    Q1 = df["Age"].quantile(0.25)
    Q3 = df["Age"].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print("When age and credit score is printed below lower score: ", lower, "and upper score: ", upper)
    df_outlier = df["Age"][(df["Age"] > upper)]
    df["Age"][df_outlier.index] = upper
    
    
    # Kredit Score
    
    Q1 = df["CreditScore"].quantile(0.25)
    Q3 = df["CreditScore"].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print("When age and credit score is printed above lower score: ", lower, "and upper score: ", upper)
    df_outlier = df["CreditScore"][(df["CreditScore"] < lower)]
    df["CreditScore"][df_outlier.index] = lower
    
    return df

In [ ]:
def outlier_process(df):
    #show_outliers(df = df)
    df_outlier = lof_observation(df = df)
    df_outlier = clear_outliers(df = df_outlier)
    return df_outlier

## 3.3. Data Analysis

### 3.3.1. Dependent variable distribution

In [ ]:
def show_dependent_variable(df):
#     sns.countplot(data = df, x = dependent_variable_name, label = 'Count') \
#     .set_title(dependent_variable_name + ' dependent variable situation', fontsize = 18, color = 'r')
    fig, axarr = plt.subplots(2, 3, figsize=(18, 6))
    sns.countplot(x = 'Geography', hue = 'Exited',data = df, ax = axarr[0][0])
    sns.countplot(x = 'Gender', hue = 'Exited',data = df, ax = axarr[0][1])
    sns.countplot(x = 'HasCrCard', hue = 'Exited',data = df, ax = axarr[0][2])
    sns.countplot(x = 'IsActiveMember', hue = 'Exited',data = df, ax = axarr[1][0])
    sns.countplot(x = 'NumOfProducts', hue = 'Exited',data = df, ax = axarr[1][1])
    sns.countplot(x = 'Tenure', hue = 'Exited',data = df, ax = axarr[1][2])
    zero, one = df[dependent_variable_name].value_counts()
    print("Dependent variable distribution;")
    print(dependent_variable_name + " 0 count:", zero)
    print(dependent_variable_name + " 1 count:", one)

### 3.3.2. Numeric columns distribution observation

In [ ]:
def show_numeric_columns_distributions(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num_cols = df.select_dtypes(include=numerics)
    columns = df_num_cols.columns[: len(df_num_cols.columns)]
    fig = plt.figure()
    fig.set_size_inches(18, 15)
    #plt.subplots(figsize=(22,22))
    length = len(columns)
    for i,j in itertools.zip_longest(columns, range(length)):
        plt.subplot((length / 2), 3, j+1)
        plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
        df_num_cols[i].hist(bins = 20, edgecolor = 'black')
        plt.title(i)
    fig = fig.suptitle('Structures of numeric variables', color = 'r' ,fontsize = 18)
    plt.show()

### 3.3.3. Status of other variables according to dependent variable

In [ ]:
def show_dependent_variable_cross_others_distributions(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_dependent_var = df[df[dependent_variable_name] == 1]
    df_num_cols = df_dependent_var.select_dtypes(include = numerics)
    columns = df_num_cols.columns[: len(df_num_cols.columns)]
    
    fig = plt.figure()
    fig.set_size_inches(18, 15)
    length = len(columns)
    for i,j in itertools.zip_longest(columns, range(length)):
        plt.subplot((length / 2), 3, j+1)
        plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
        df_num_cols[i].hist(bins = 20, edgecolor = 'black')
        plt.title(i)
    fig = fig.suptitle(dependent_variable_name + ' Status of other variables according to 1 dependent variable', color = 'r', fontsize = 18)
    plt.show()

### 3.3.4. Categorical variables are observed according to the dependent variable

In [ ]:
def show_dependent_variable_cross_categorical_distributions(df, categorical_columns):
    sns.set(font_scale = 0.7) 
    fig, axes = plt.subplots(nrows = int( len(categorical_columns) / 2 ) , ncols = 2, figsize = (7,9))
    fig.tight_layout()
    for ax,col in zip(axes.flatten(), categorical_columns):
        sns.countplot(x = df[col], hue = dependent_variable_name, data = df, ax = ax)
    fig.suptitle('Categorical variables are monitored according to the dependent variable', color = 'r', fontsize = 1)

### 3.3.5. The main method that started all data analysis

In [ ]:
def show_data_analysis(df):
    show_dependent_variable(df)
    show_numeric_columns_distributions(df)
    show_dependent_variable_cross_others_distributions(df)
    show_dependent_variable_cross_categorical_distributions(df = df_outlier, categorical_columns = ["Gender","Geography","HasCrCard","IsActiveMember"])

## 4. Feature Engineering

### 4.1. Credit Score grouping ( min= 358 and max= 800)

In [ ]:
def credit_score_table(row):
    
    credit_score = row.CreditScore
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 851:
        return "Top"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score < 300:
        return "Deep"

### 4.2. Product utilization RATE by YEAR

In [ ]:
def product_utilization_rate_by_year(row):
    number_of_products = row.NumOfProducts
    tenure = row.Tenure
    
    if number_of_products == 0:
        return 0
    
    if tenure == 0:
        return number_of_products
    
    rate = number_of_products / tenure
    return rate

### 4.3. Product utilization rate by estimated SALARY

In [ ]:
def product_utilization_rate_by_estimated_salary(row):
    number_of_products = row.number_of_products
    estimated_salary = row.EstimatedSalary
    
    if number_of_products == 0:
        return 0

    
    rate = number_of_products / estimated_salary
    return rate

### 4.4. According to countries monthly average salaries

In [ ]:
def countries_monthly_average_salaries(row):
    #brutto datas from  https://tr.wikipedia.org/wiki/Aylık_ortalama_ücretlerine_göre_Avrupa_ülkeleri_listesi
    fr = 3696    
    de = 4740
    sp = 2257
    salary = row.EstimatedSalary / 12
    country = row.Geography              # Germany, France and Spain
    
    if country == 'Germany':
        return salary / de
    elif country == "France":
        return salary / fr
    elif country == "Spain": 
        return salary / sp

### 4.5. The main method that started all Feature Engineering

In [ ]:
def feature_engineering(df, is_show_graph = False):
    df_fe = df.copy()
    
    #bakiye_maas_orani
    balance_salary_rate = 'balance_salary_rate'
    df_fe[balance_salary_rate] = df_fe.Balance / df_fe.EstimatedSalary
    
    #yila_gore_urun_kullanim_orani
    df_fe = df_fe.assign(product_utilization_rate_by_year=df_fe.apply(lambda x: product_utilization_rate_by_year(x), axis=1)) 
    
    #tahmini_maasa_gore_urun_kullanim_orani
    #df_fe = df_fe.assign(product_utilization_rate_by_estimated_salary = df_fe.apply(lambda x: product_utilization_rate_by_estimated_salary(x), axis=1)) 
    
    
    #musteri_yilina göre yaşa göre standardize edilmesi, oranlanmasi - ergenlik donemini cikariyoruz!
    tenure_rate_by_age = 'tenure_rate_by_age'
    df_fe[tenure_rate_by_age] = df_fe.Tenure / (df_fe.Age-17)
    
    #yaşa göre kredi_skoru standardize edilmesi, oranlanmasi - ergenlik donemini cikariyoruz!
    credit_score_rate_by_age = 'credit_score_rate_by_age'
    df_fe[credit_score_rate_by_age] = df_fe.CreditScore / (df_fe.Age-17)
    
    #maaşa gore kullanilan urun oran, oranlanmasi
    product_utilization_rate_by_salary = 'product_utilization_rate_by_salary'
    #df_fe[product_utilization_rate_by_salary] = df_fe.Tenure / (df_fe.EstimatedSalary)
   
    #maaşa göre kredi_skoru urun oran, oranlanmasi
    credit_score_rate_by_salary = 'credit_score_rate_by_salary'
    df_fe[credit_score_rate_by_salary] = df_fe.CreditScore / (df_fe.EstimatedSalary)
    
    #Feature Eng. oluşturulan değişkenlerin bağımlı değişkene göre grafikleri gösterilsin mi?
    if is_show_graph:
        fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (20,12))
        fig.tight_layout()
        sns.boxplot(y = balance_salary_rate, x = dependent_variable_name, hue = dependent_variable_name, data = df_fe, ax = axes[0][0])
        sns.boxplot(y = product_utilization_rate_by_year, x = dependent_variable_name, hue = dependent_variable_name, data = df_fe, ax = axes[0][1])
        #sns.countplot(x = credit_score_rate_by_age, hue = dependent_variable_name, data = df_fe, ax = axes[1][0])
        #sns.countplot(x = credit_score_rate_by_age, hue = dependent_variable_name, data = df_fe, ax = axes[1][1])
        plt.ylim(-1, 5)
    
    
    #feature engineering add- kredi_skor_tablosu
    df_fe = df_fe.assign(credit_score_table=df_fe.apply(lambda x: credit_score_table(x), axis=1))
    
    #feature engineering add- ulkelere ortalama  maas durum
    df_fe = df_fe.assign(countries_monthly_average_salaries = df_fe.apply(lambda x: countries_monthly_average_salaries(x), axis=1)) 
    
    return df_fe

## 5. Data Encoding

In [ ]:
# vucut_yag_orani, vucut_yag_orani_kategori,beden_kitle_kategori,insulin_kategori,kucuk_tansiyon_kategori

def data_encoding(df):
    df_model = df.copy()
    '''
    # It was attempted to reduce the number of 0 observations.
    churn_zero = df_model[df_model.apply(lambda x: True if x['Exited'] == 0 else False , axis=1)]
    df_train = churn_zero.sample(frac=0.1,random_state=100)
    df_model = df_model.drop(df_train.index)
    '''
    
    
    # >>>> Categorical columns <<<<<
    
    non_encoding_columns = ["Geography","HasCrCard","IsActiveMember","Gender","NumOfProducts","Tenure","credit_score_table"]
    
    df_non_encoding = df_model[non_encoding_columns]
    df_model = df_model.drop(non_encoding_columns,axis=1)
    
    
    df_encoding = df_non_encoding.copy()
    
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    df_encoding["gender_category"] = encoder.fit_transform(df_non_encoding.Gender)
    df_encoding["country_category"] = encoder.fit_transform(df_non_encoding.Geography)
    df_encoding["credit_score_category"] = encoder.fit_transform(df_non_encoding.credit_score_table)

    

    df_encoding.reset_index(drop=True, inplace=True)
    df_model.reset_index(drop=True, inplace=True)
    df_model = pd.concat([df_model,df_encoding],axis=1)

    df_model = df_model.drop(["Geography","Gender","CustomerId","Surname","credit_score_table","CreditScore","EstimatedSalary"],axis=1)
    df_model = df_model.reset_index()
    df_model = df_model.drop('index',axis=1)
    
    df_model.loc[df_model.HasCrCard == 0, 'credit_card_situation'] = -1
    df_model.loc[df_model.IsActiveMember == 0, 'is_active_member'] = -1
    return df_model

## 6. Model Preparation

In [ ]:
def model_prepare(df_model):
    y = df_model[dependent_variable_name]
    X = df_model.loc[:, df_model.columns != dependent_variable_name]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12345)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform (X_test)
    return X_train, X_test, y_train, y_test

### 6.1 Part of Data TRAIN

In [ ]:
def data_training(X_train, X_test, y_train, y_test):

    models = []
    models.append(('LOGR',LogisticRegression()))
    models.append(('KNN',KNeighborsClassifier()))
    models.append(('CART',DecisionTreeClassifier()))
    models.append(('RF',RandomForestClassifier()))
    #models.append(('SVC',SVC()))
    models.append(('GBM',GradientBoostingClassifier()))
    models.append(('XGBoost',XGBClassifier()))
    models.append(('LightGBM',LGBMClassifier()))
    models.append(('CatBoost',CatBoostClassifier()))

    df_result = pd.DataFrame(columns=["model","accuracy_score","scale_method","0_precision","0_recall","1_precision","1_recall"])
    index = 0
    for name,model in models:
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        score = accuracy_score(y_test,y_pred)
        class_report = classification_report(y_test,y_pred,digits=2,output_dict=True)
        zero_report = class_report['0']
        one_report = class_report['1']
        df_result.at[index,['model','accuracy_score','scale_method',"0_precision","0_recall","1_precision","1_recall"]] = [name,score,"NA",zero_report['precision'],zero_report['recall'],one_report['precision'],one_report['recall']]
        index += 1
    return df_result.sort_values("accuracy_score",ascending=False)

## 7. HELPing Functions

In [ ]:
# Function to give best model score and parameters

def best_model(model):
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)
    
def get_auc_scores(y_actual, method,method2):
    auc_score = roc_auc_score(y_actual, method); 
    fpr_df, tpr_df, _ = roc_curve(y_actual, method2); 
    return (auc_score, fpr_df, tpr_df)


from matplotlib import rc,rcParams
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.rcParams.update({'font.size': 16})
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, color="blue")
    plt.yticks(tick_marks, classes, color="blue")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="red" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

''' inf yakalama
indices_to_keep = df_encoded.isin([np.nan, np.inf, -np.inf]).any(1)
df_encoded[indices_to_keep].astype(np.float64)
'''

## 8. Data Cleaning

In [ ]:
df_prep = data_prepare()

df_outlier = outlier_process(df = df_prep)


show_data_analysis(df_prep)
show_outliers(df = df_outlier)

+ Let's see after feature engineering:

In [ ]:
df_fe = feature_engineering(df = df_outlier)
df_fe.head(3)

+ And see after data encoding:

In [ ]:
df_encoded = data_encoding(df_fe)
df_encoded.drop(['credit_card_situation', 'is_active_member'], axis=1, inplace=True)
df_encoded.head(3)

## Now, let's see correlation graph:

In [ ]:
correlation = df_encoded.corr().abs()
plt.figure(figsize=(15,15))
sns.heatmap(correlation, annot=True)
plt.show()

## List of correlation scores:

In [ ]:
corrs_results = df_encoded.corrwith(df_encoded["Exited"]).abs().nlargest(24)
corrs_results

# 9. Apply Model

In [ ]:
# model_prepare test, train split 0.2
X_train, X_test, y_train, y_test = model_prepare(df_model = df_encoded)

### 9.1. Logistic Regression Model

+ Have a look with Logistic Regression.

In [ ]:
logr_model = LogisticRegression().fit(X_train,y_train)
y_pred = logr_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred, digits=4))
print("Accuracy score of Logistic Regression: ", accuracy)

### 9.2. Model Training

+ This is for all LogisticRegression, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier, and LGBMClassifier model objects:

In [ ]:
training_result = data_training(X_train, X_test, y_train, y_test)
training_result

+ Now, let's have a look which model is best for us:

In [ ]:
training_result

# 10. Model Tunning

## 10.1. XGBoost Tuning

In [ ]:
xgb_model=XGBClassifier(silent=0, learning_rate=0.23, max_delta_step=5,
                            objective='reg:logistic',n_estimators=92, 
                            max_depth=5, eval_metric="logloss", gamma=3,base_score=0.5)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of Tuned XGBoost Regression: ", accuracy_score(y_test, y_pred))

## 10.2. Random Forest Tuning

In [ ]:
param_grid = {'max_depth': [3, 5, 6, 7, 8], 'max_features': [2,4,6,7,8,9],'n_estimators' : [50,100], 'min_samples_split': [3, 5, 6, 7]}
randFor_grid = GridSearchCV(RandomForestClassifier(), param_grid, cv = 5, refit = True, verbose = 0)
randFor_grid.fit(X_train,y_train)
best_model(randFor_grid)

+ Using the parameters and get final version accuracy score.

In [ ]:
rnd_model = RandomForestClassifier(max_depth=8, max_features=6, min_samples_split=6,n_estimators=50)
rnd_model.fit(X_train, y_train)
y_pred = rnd_model.predict(X_test)
print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of tuned Random Forest model: ", accuracy_score(y_test, y_pred))

## 10.3. LightGBM Tuning

In [ ]:
lgbm_model = LGBMClassifier(silent = 0, learning_rate = 0.09, max_delta_step = 2, n_estimators = 100, boosting_type = 'gbdt',
                            max_depth = 10, eval_metric = "logloss", gamma = 3, base_score = 0.5)
lgbm_model.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_test)
print(classification_report(y_test, y_pred, digits=2))
print("Accuracy score of tuned LightGBM model: ", accuracy_score(y_test, y_pred))

# 11. Confussion Matrix

In [ ]:
cfm = confusion_matrix(y_test, y_pred=y_pred)
plot_confusion_matrix(cfm, classes=['Non Churn','Churn'],
                      title='Churn Confusion matrix')


tn, fp, fn, tp = cfm.ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

# 12. ROC Curve

In [ ]:
import scikitplot as skplt
y_pred_proba = lgbm_model.predict_proba(X_test)
skplt.metrics.plot_roc_curve(y_test, y_pred_proba, figsize=(8,8))

plt.show()

# 13. Feature Importance

In [ ]:
feature_index = df_encoded.loc[:, df_encoded.columns != dependent_variable_name]

feature_importance = pd.Series(lgbm_model.feature_importances_, 
                               index=feature_index.columns).sort_values(ascending=False)
sns.barplot(x = feature_importance, y = feature_importance.index, color='r', saturation=1)
plt.xlabel('Variable Severity Scores')
plt.ylabel('Variables')
plt.title('Variable Severity Levels')
plt.show()

# 14. Conclusion

+ Tujuan kami dalam proyek ini adalah mengembangkan model prediksi churn menggunakan algoritme pembelajaran mesin. 
+ Ada 10.000 baris dalam kumpulan data dan tidak ada nilai yang hilang. dan dataset terdiri dari 13 variabel. 
+ Kesimpulan berikut didapat dari analisis fitur: 
    + Sebagian besar nasabah yang menggunakan produk 3 dan 4 berhenti bekerja sama dengan bank. 
    + Bahkan, semua pelanggan yang menggunakan produk nomor 4 sudah tidak ada. 
    + Pelanggan berusia antara 40 dan 65 tahun lebih cenderung keluar dari bank. 
    + Mereka yang memiliki skor kredit di bawah 450 memiliki tingkat pengabaian yang tinggi. 
    + Prediksi dilakukan dengan total 8 model klasifikasi. 
    + Head tertinggi diambil dengan metode LightGBM. 
    + Skor akurasi dan validasi silang dihitung untuk setiap model dan hasilnya ditampilkan.